1. set up dictionary for the corpus
2. token2id, id2token
3. extract questions and answers: question mark; and sentences less than 20 words
4. word2vec

convert to lower case; but no expansion
try not converting lower case

In [1]:
from gensim import corpora,models
import os
import chardet   #需要导入这个模块，检测编码格式
import re
import numpy as np
import pickle 
import pandas as pd
import gensim

In [2]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [3]:
# 获取单词的词性
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [4]:
from smart_open import smart_open

def sentence_extract(line):
    try:
        encode_type = chardet.detect(line)  
        line = line.decode(encode_type['encoding']) #进行相应解码，赋给原标识符（变量）
        line = line.lower().split('<eos>')
        if len(line)<2:
            return []
    #     line = line.replace('<EOS>','\n').split('\n')
        line = list(filter(None,line))[:2]
        line = [x.split() for x in line]
        diag_list = [line] if (max([len(x) for x in line]) <= 20)&(min([len(x) for x in line]) >= 3) else []
    except BaseException:
        diag_list = []
    return diag_list

class MyCorpus(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for line in smart_open(self.dirname, 'rb'):
            yield from sentence_extract(line)

file_dir = '/Users/yan/Documents/document/EPFL/MA2/semesterprj/code/session_segmentation.txt'
corpus = MyCorpus(file_dir)
sentences = [item for sublist in corpus for item in sublist]    


In [5]:
# import pandas as pd
# import nltk
# from nltk.tokenize import RegexpTokenizer
# def read_lines(dirname):
#     line_dict = {}
#     with open(dirname, 'r', encoding='iso-8859-1') as f:  # TODO: Solve Iso encoding pb !
#         tokenizer = RegexpTokenizer(r'\w+')
#         for line in f:
#             line = line.replace('[','').replace(']','')
#             tmp = line.split('\n')[0].split(" +++$+++ ")
#             line_ = nltk.word_tokenize(tmp[-1])
# #             line_ = tokenizer.tokenize(tmp[-1].lower())
# #             line_ = [x for x in line_ if x not in stop]
#             line_dict[tmp[0]] = {'ID1':tmp[1],'MovieID':tmp[2],'Name':tmp[3],'Line':line_}
#         return line_dict
# movie_lines = read_lines('../datasets/cornell-corpus/movie_lines.txt')
# cornell = pd.read_csv('../datasets/cornell-corpus/cleaned.csv')


# def sentence_extract(IDs):
#     try:
#         lines = []
#         for ID in IDs:
#             line = movie_lines[ID]['Line']
#             line = [x.lower().lstrip().rstrip() for x in line]
            
#             lines.append(line)
#         diag_list = [lines] if max([len(x) for x in lines]) > 20 else []
            
#     except BaseException:
#         diag_list = []
#     return diag_list

# class MyCorpus(object):
#     def __init__(self, df):
#         self.df = df
 
#     def __iter__(self):
#         for i in range(len(self.df)):
#             yield from sentence_extract(self.df.LineIDs.loc[i].split(', ')[:2])

# corpus = MyCorpus(cornell)
# sentences = [item for sublist in corpus for item in sublist]    
# sentences = list(filter(None,sentences))

---

In [6]:
words_all = [item for sublist in sentences for item in sublist]

In [7]:
tf_ = pd.DataFrame(words_all,columns=['word']).word.value_counts()/len(words_all)

In [8]:
tf_.describe()

count    2.123100e+04
mean     4.710094e-05
std      8.895366e-04
min      8.586389e-07
25%      8.586389e-07
50%      2.575917e-06
75%      6.869111e-06
max      6.593659e-02
Name: word, dtype: float64

In [9]:
words = list(set(words_all))
len(words)

21231

In [10]:
tagged_sent = pos_tag(words)

wnl = WordNetLemmatizer()
lemmas = {}
for tag in tagged_sent:
    wordnet_pos = get_wordnet_pos(tag[1]) or wordnet.NOUN
    lemmas.update({tag[0]:wnl.lemmatize(tag[0], pos=wordnet_pos)})

In [11]:
# word number after lemmatizer
len(set(lemmas.values()))

17257

In [12]:
model_opst = gensim.models.Word2Vec(iter=1,size=300,min_count=1)  # an empty model, no training yet
model_opst.build_vocab(sentences)  # can be a non-repeatable, 1-pass generator

model_opst.intersect_word2vec_format('GoogleNews-vectors-negative300.bin.gz',
                                lockf=1.0, # allow further training updates of merged vectors
                                binary=True)


In [13]:
model_opst.train(sentences,total_examples=model_opst.corpus_count,epochs=model_opst.epochs)

(718170, 1164634)

In [14]:
corpus_dict = corpora.Dictionary([words])
corpus = [corpus_dict.doc2bow(text) for text in [words]]
print(corpus_dict)

Dictionary(21231 unique tokens: ['!', '#', '$', '%', "'"]...)


In [15]:
corpus_dict[0]
id2token = corpus_dict.id2token
token2id = corpus_dict.token2id

In [16]:
# pickle.dump(VAD_dict,open('./pre-data/VAD_dict.pickle','wb'))

In [17]:
VAD_extend = pd.read_csv('./seq2seq_attn/affect-rich/VAD_extend_clean.csv')
VAD_extend.dropna(subset=['Word'],inplace=True)
VAD_extend.set_index('Word',inplace=True)

lambda_ = 0.1
VAD_extend = VAD_extend.clip(lower=3,upper=7) - [5,3,5]
VAD_extend = VAD_extend.mul(lambda_)

VAD_extend_dict = VAD_extend.to_dict('index')

In [18]:
len(VAD_extend)

26959

In [19]:
# number of lemmas in VAD dataframe (after extension)
sum([lemma in VAD_extend_dict.keys() for lemma in lemmas.values()])

12639

In [20]:
VAD_extend.describe()

,V,A,D
count,26959.000000,26959.000000,26959.000000
mean,0.006111,0.122721,0.018835
std,0.107662,0.075985,0.082669
min,-0.200000,0.000000,-0.200000
25%,-0.065000,0.070000,-0.032000
50%,0.016667,0.115000,0.026000
75%,0.080679,0.165389,0.074333
max,0.200000,0.400000,0.200000


---

In [21]:
word_embeddings = []
VAD_list = []
tf = []
for i in range(len(corpus_dict)):
    embedding = list(model_opst.wv[id2token[i]])
    word_lemma = lemmas[id2token[i]]
    vad = list(VAD_extend_dict[word_lemma].values()) if word_lemma in VAD_extend_dict.keys() else [0,0,0]
    # after normalization for words outside dictionary the VAD value should be [0,0,0]
    word_embeddings.append(np.array(embedding))
    VAD_list.append(vad)
    tf.append(tf_.loc[id2token[i]])

In [25]:
# create random vector for <go> and <eos>
word_embeddings.append(np.random.randn(model_opst.vector_size))
VAD_list.append([0,0,0])
tf.append(len(sentences)/len(words_all))
id2token[len(id2token)] = '<go>'
token2id['<go>'] = len(id2token)-1

word_embeddings.append(np.random.randn(model_opst.vector_size))
VAD_list.append([0,0,0])
tf.append(len(sentences)/len(words_all))
id2token[len(id2token)] = '<eos>'
token2id['<eos>'] = len(id2token)-1

In [26]:
word_embeddings = np.array(word_embeddings)

In [27]:
VAD = np.array(VAD_list)

In [28]:
tf = np.array(tf)

In [29]:
pd.DataFrame(VAD).describe()

,0,1,2
count,21233.000000,21233.000000,21233.000000
mean,0.011576,0.074317,0.017877
std,0.090199,0.091019,0.072521
min,-0.200000,0.000000,-0.200000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.035333,0.000000
75%,0.053000,0.130000,0.057000
max,0.200000,0.400000,0.200000


In [30]:
save_dir = '/Users/yan/Documents/document/EPFL/MA2/semesterprj/code/seq2seq_attn/affect-rich/input/'
np.save(save_dir+'word_embeddings.npy',word_embeddings)

pickle.dump(id2token,open(save_dir+'id2token.pickle','wb'))
pickle.dump(token2id,open(save_dir+'token2id.pickle','wb'))

In [31]:
sentence_token2id = []
for i in range(len(sentences)):
    sentence_token2id.append([token2id.get(item) for item in sentences[i]])

In [32]:
# sentence_token2id = [[token2id['<go>']]+x+[token2id['<eos>']] for x in sentence_token2id]

In [33]:
enc_input = sentence_token2id[::2]
dec_input = [[token2id['<go>']]+x for x in sentence_token2id[1::2]] # '<go>' + 
target = [x+[token2id['<eos>']] for x in sentence_token2id[1::2]] # + '<eos>'

In [34]:
max_uttr_len_enc = max([len(x) for x in enc_input])
max_uttr_len_dec = max([len(x) for x in dec_input])
max_uttr_len_target = max([len(x) for x in target])
print(max_uttr_len_enc,max_uttr_len_dec,max_uttr_len_target)

20 21 21


In [35]:
enc_input = np.array([i + [0]*(max_uttr_len_enc-len(i)) for i in enc_input])
dec_input = np.array([i + [0]*(max_uttr_len_dec-len(i)) for i in dec_input])
target = np.array([i + [0]*(max_uttr_len_target-len(i)) for i in target])

In [36]:
from sklearn.model_selection import train_test_split
enc_train, enc_test, dec_train, dec_test, target_train, target_test = train_test_split(
    enc_input, dec_input, target, test_size=0.3, random_state=1)

enc_test, enc_val, dec_test, dec_val,target_test,target_val = train_test_split(
    enc_test, dec_test, target_test, test_size=0.5, random_state=1)


In [37]:
enc_input.shape

(66207, 20)

In [38]:
enc_train.shape

(46344, 20)

In [39]:
enc_test.shape

(9931, 20)

---
incorporate VAD embedding of words into cross-entropy loss

In [ ]:
delta = 0.15

In [ ]:
VAD.shape

In [ ]:
V = VAD.shape[0]

In [ ]:
VAD_loss = 1 + delta*np.linalg.norm(VAD,axis=1)

In [ ]:
VAD_loss = V*VAD_loss/sum(VAD_loss)

In [ ]:
sum(1 + delta*np.linalg.norm(VAD,axis=1))

In [ ]:
VAD_loss.shape

---
$\mu(x_t)$
- uniform importance
- global importance
- local importance

In [ ]:
# uniform importance
mu_ui = np.ones(tf.shape[0])

In [ ]:
# globale importance
a = 0.001
mu_gi = a/(a+tf)

In [ ]:
mu_gi.max()/mu_gi.min()

In [ ]:
# local importance
epsilon = 10e-8
mu_li = np.log(1/tf+epsilon)/sum(np.log(1/tf+epsilon))

In [ ]:
mu_li.max()/mu_li.min()

---
saving

dec_input: `<go>` + answer
target: answer + `<eos>`

In [ ]:
np.save(save_dir+'VAD.npy',VAD)
np.save(save_dir+'tf.npy',tf)
np.save(save_dir+'VAD_loss.npy',VAD_loss)

np.save(save_dir+'mu_ui.npy',mu_ui)
np.save(save_dir+'mu_gi.npy',mu_gi)
np.save(save_dir+'mu_li.npy',mu_li)

In [40]:
enc_input_len = [sum(x!=0) for x in enc_train]
dec_input_len = [sum(x!=0) for x in dec_train]
np.save(save_dir+'/train/enc_input.npy',enc_train)
np.save(save_dir+'/train/dec_input.npy',dec_train)
np.save(save_dir+'/train/target.npy',target_train)
np.save(save_dir+'/train/enc_input_len.npy',enc_input_len)
np.save(save_dir+'/train/dec_input_len.npy',dec_input_len)

enc_input_len = [sum(x!=0) for x in enc_val]
dec_input_len = [sum(x!=0) for x in dec_val]
np.save(save_dir+'/validation/enc_input.npy',enc_val)
np.save(save_dir+'/validation/dec_input.npy',dec_val)
np.save(save_dir+'/validation/target.npy',target_val)
np.save(save_dir+'/validation/enc_input_len.npy',enc_input_len)
np.save(save_dir+'/validation/dec_input_len.npy',dec_input_len)


In [41]:
enc_input_len = [sum(x!=0) for x in enc_test]
dec_input_len = [sum(x!=0) for x in dec_test]
np.save(save_dir+'/test/enc_input.npy',enc_test)
np.save(save_dir+'/test/dec_input.npy',dec_test)
np.save(save_dir+'/test/target.npy',target_test)
np.save(save_dir+'/test/enc_input_len.npy',enc_input_len)
np.save(save_dir+'/test/dec_input_len.npy',dec_input_len)
